# If you're going to change something in this notebook after running it, restart the kernel - otherwise, it'll do it for you

## Imports

In [1]:
import os, sys, glob, tables
import numpy as np
# pip install simweights <--- run this once to install the package
# then replace <username> with your username and <version> with your Python version
sys.path.append('/home/tfutrell/.local/lib/python3.12/site-packages')
import simweights

## Sim folders:

In [2]:
'''
2012:
    EPOS:
        Fe:
            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/EPOS-LHC/Fe/12635_v1s/h5files/*.h5
            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/EPOS-LHC/Fe/20197_v1s/h5files/*.h5
        p:
            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/EPOS-LHC/p/12634_v1s/h5files/*.h5
            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/EPOS-LHC/p/12634_v1s/h5files/*.h5
            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/EPOS-LHC/p/20196_v1s/h5files/*.h5
    SIBYLL2.1:
        Fe:
            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/SIBYLL2.1/Fe/12362_v1s/h5files/*.h5
            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/SIBYLL2.1/Fe/20144_v1s/h5files/*.h5
        He:
            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/SIBYLL2.1/He/12630_v1s/h5files/*.h5
            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/SIBYLL2.1/He/20145_v1s/h5files/*.h5
        O:
            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/SIBYLL2.1/O/12631_v1s/h5files/*.h5
            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/SIBYLL2.1/O/20146_v1s/h5files/*.h5
        p:
            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/SIBYLL2.1/p/12360_v1s/h5files/*.h5
            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/SIBYLL2.1/p/20143_v1s/h5files/*.h5
    SIBYLL2.3:
        /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/SIBYLL2.3/Fe/12633_v1s/h5files/*.h5
        /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/SIBYLL2.3/p/12632_v1s/h5files/*.h5
2015:
    SIBYLL2.1:
        '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2015/SIBYLL2.1/p/20174_v1s/h5files/*.h5'
        '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2015/SIBYLL2.1/He/20178_v1s/h5files/*.h5'
        '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2015/SIBYLL2.1/O/20179_v1s/h5files/*.h5'
        '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2015/SIBYLL2.1/Fe/20180_v1s/h5files/*.h5'
2018:
    EPOS:
        '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/EPOS-LHC/Fe/23201_v3/h5files/*.h5'
        '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/EPOS-LHC/He/23199_v3/h5files/*.h5'
        '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/EPOS-LHC/O/23200_v3/h5files/*.h5'
        '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/EPOS-LHC/p/23198_v3/h5files/*.h5'
    SIBYLL2.3d:
        Fe:
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Fe/22585_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Fe/22586_v3//h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Fe/22888_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Fe/22891_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Fe/22892_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Fe/22893_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Fe/22894_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Fe/22943_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Fe/22976_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Fe/22977_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Fe/22978_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Fe/22979_v3/h5files/*.h5'
        He:
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/He/22579_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/He/22580_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/He/22922_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/He/22923_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/He/22924_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/He/22925_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/He/22926_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/He/22941_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/He/22966_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/He/22967_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/He/22968_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/He/22969_v3/h5files/*.h5'
        O:
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22581_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22583_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22932_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22942_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22970_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22971_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22974_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22975_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22981_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22982_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22984_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22985_v3/h5files/*.h5'
        p:
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22569_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22570_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22899_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22900_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22919_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22920_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22921_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22940_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22962_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22963_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22964_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22965_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/23074_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/23080_v3/h5files/*.h5'
            '/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/23094_v3/h5files/*.h5'
'''

"\n2012:\n    EPOS:\n        Fe:\n            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/EPOS-LHC/Fe/12635_v1s/h5files/*.h5\n            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/EPOS-LHC/Fe/20197_v1s/h5files/*.h5\n        p:\n            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/EPOS-LHC/p/12634_v1s/h5files/*.h5\n            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/EPOS-LHC/p/12634_v1s/h5files/*.h5\n            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/EPOS-LHC/p/20196_v1s/h5files/*.h5\n    SIBYLL2.1:\n        Fe:\n            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/SIBYLL2.1/Fe/12362_v1s/h5files/*.h5\n            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/SIBYLL2.1/Fe/20144_v1s/h5files/*.h5\n        He:\n            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/SIBYLL2.1/He/12630_v1s/h5files/*.h5\n            /data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/SIBYLL2.1/He/20145_v1s/h5files/*.h5\n        O:\n            /data/ana/CosmicRay/Ice

## Set up weighter

In [3]:
weighter = None
print('Getting iron weights...')
fe_list = glob.glob('/data/ana/CosmicRay/IceTop_level3/sim/IC86.2015/SIBYLL2.1/Fe/20180_v1s/h5files/*.h5')

for filename in fe_list:
    file_obj = tables.open_file(filename, 'r')
    
    if weighter is None:
        weighter = simweights.IceTopWeighter(file_obj)
    else:
        weighter += simweights.IceTopWeighter(file_obj)

print('Got iron weights!')        

Getting iron weights...
Got iron weights!


In [4]:
print('Getting helium weights...')
he_list = glob.glob('/data/ana/CosmicRay/IceTop_level3/sim/IC86.2015/SIBYLL2.1/He/20178_v1s/h5files/*.h5')

for filename in he_list:
    file_obj = tables.open_file(filename, 'r')
    weighter += simweights.IceTopWeighter(file_obj)

print('Got helium weights!')

Getting helium weights...
Got helium weights!


In [5]:
print('Getting oxygen weights...')
o_list = glob.glob('/data/ana/CosmicRay/IceTop_level3/sim/IC86.2015/SIBYLL2.1/O/20179_v1s/h5files/*.h5')
    
    
for filename in o_list:
    file_obj = tables.open_file(filename, 'r')
    weighter += simweights.IceTopWeighter(file_obj)

print('Got oxygen weights!')

Getting oxygen weights...
Got oxygen weights!


In [6]:
print('Getting proton weights...')
p_list = glob.glob('/data/ana/CosmicRay/IceTop_level3/sim/IC86.2015/SIBYLL2.1/p/20174_v1s/h5files/*.h5')

for filename in p_list:
    file_obj = tables.open_file(filename, 'r')
    weighter += simweights.IceTopWeighter(file_obj)

print('Got proton weights!')

Getting proton weights...
Got proton weights!


## Save data

In [7]:
# Save energy
if not os.path.isfile('saved_data/energy.npy'):
    print('Saving energy...')
    with open('saved_data/energy.npy', 'wb') as f:
        np.save(f, weighter.get_column('MCPrimary', 'energy'))
    print('Saved!')
else:
    print('File already exists, skipping')

Saving energy...
Saved!


In [8]:
# Save particle type
if not os.path.isfile('saved_data/particle_type.npy'):
    print('Saving particle type...')
    with open('saved_data/particle_type.npy', 'wb') as f:
        np.save(f, weighter.get_column('MCPrimary', 'type'))
    print('Saved!')
else:
    print('File already exists, skipping')

Saving particle type...
Saved!


In [9]:
# Save H4a weights
if not os.path.isfile('saved_data/Hweights.npy'):
    print('Saving H4a weights...')
    with open('saved_data/Hweights.npy', 'wb') as f:
        np.save(f, weighter.get_weights(simweights.GaisserH4a_IT()))
    print('Saved!')
else:
    print('File already exists, skipping')

Saving H4a weights...
Saved!


In [10]:
# Save GSF weights
if not os.path.isfile('saved_data/Gweights.npy'):
    print('Saving GSF weights...')
    with open('saved_data/Gweights.npy', 'wb') as f:
        np.save(f, weighter.get_weights(simweights.GlobalSplineFit_IT()))
    print('Saved!')
else:
    print('File already exists, skipping')

Saving GSF weights...
Saved!


In [11]:
# Save hits
if not os.path.isfile('saved_data/hits.npy'):
    print('Saving hits...')
    with open('saved_data/hits.npy', 'wb') as f:
        np.save(f, weighter.get_column('IceTopHLCSeedRTPulses_SnowUnAttenuated_info', 'nstrings').astype(int))
    print('Saved!')
else:
    print('File already exists, skipping')

Saving hits...
Saved!


In [12]:
# Save reconstruction passed cut
if not os.path.isfile('saved_data/reco_pass.npy'):
    print('Saving reco_pass...')
    with open('saved_data/reco_pass.npy', 'wb') as f:
        np.save(f, weighter.get_column('IT73AnalysisIceTopQualityCuts', 'IceTop_reco_succeeded').astype(bool))
    print('Saved!')
else:
    print('File already exists, skipping')

Saving reco_pass...
Saved!


In [13]:
# Save zenith cut
if not os.path.isfile('saved_data/_cut.npy'):
    print('Saving zenith cut...')
    with open('saved_data/zenith_cut.npy', 'wb') as f:
        np.save(f, weighter.get_column('MCPrimary', 'zenith') < 0.9599310886)
    print('Saved!')
else:
    print('File already exists, skipping')

Saving zenith cut...
Saved!
